In [1]:
from datasets import Dataset
import pandas as pd
import csv
import re
import os

In [2]:
model_name = 'xlm-roberta-base'

In [3]:
src_dir = '../augmenting_nlms_meco_data/sentiment/it_sentipolc'

In [4]:
sentipolc_files = {
    'train': [os.path.join(src_dir, file_name) for file_name in os.listdir(src_dir) if 'training_set' in file_name][0],
    'test': [os.path.join(src_dir, file_name) for file_name in os.listdir(src_dir) if 'test_set' in file_name][0]
}

In [5]:
sentipolc_files

{'train': '../augmenting_nlms_meco_data/sentiment/it_sentipolc/training_set_sentipolc16.csv',
 'test': '../augmenting_nlms_meco_data/sentiment/it_sentipolc/test_set_sentipolc16_gold2000.csv'}

In [11]:
def create_dataset_from_faulty_csv(src_path):
    dataset_dict = {'text': [], 'label_pos': [], 'label_neg':[]}
    with open(src_path) as src_file:
        csv_reader = csv.reader(src_file, delimiter=',', quotechar='"')
        print('')
        for row in csv_reader:
            if row[0] == 'idtwitter':
                continue
            if len(row) != 9:
                cut_row = row[:9]
                cut_row[8] += ',' + ', '.join(row[9:])
                row = cut_row
            dataset_dict['text'].append(row[8])
            dataset_dict['label_pos'].append(int(row[2]))
            dataset_dict['label_neg'].append(int(row[3]))
    return Dataset.from_dict(dataset_dict)

In [12]:
train_data = create_dataset_from_faulty_csv(sentipolc_files['train'])
test_data = create_dataset_from_faulty_csv(sentipolc_files['test'])

In [13]:
train_data, test_data

(Dataset({
     features: ['text', 'label_pos', 'label_neg'],
     num_rows: 7410
 }),
 Dataset({
     features: ['text', 'label_pos', 'label_neg'],
     num_rows: 1998
 }))

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)